## STRATEGY 

Parameters:
	Stretch:	0.5
	LongTermMALen:	200
	ShortTermMALen:	20
		
Data:
	ATRValue:	ATR(3)
	ADXValue:	ADX(5)
	
	ShortTermMA:	MA(Close, ShortTermMALen)
	LongTermMA:	MA(Close, LongTermMALen)

	ClosingRange:	(Close - Low) / (High - Low)
	IsUptrend: 	Close > LongTermMA
	IsVolatile:	ADXValue > 30			  

	Long_limit:	    Low - (ATRValue * Stretch) // limit order price
	Long_trigger:	ClosingRange < 0.3 // close in lower 30% of bar
	Long_setup:	    LongTrigger and IsUptrend and IsLiquid and IsVolatile
	
	StopLossExit:	Close < ShortTermMA



	Exit_rule:	Close > FillPrice or StopLossExit
	Exit_price:	NextOpen

In [1]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta
from numba import njit
import plotly.graph_objs as go
from vectorbt.portfolio.enums import SizeType, Direction


/Users/andre/Documents/Python_local/strategy_lab/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## data collection

In [2]:

symbols = ['ETH-USDT',"BTC-USDT"]
timeframe = "1day"
start_time = "2024-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")



# Fetch data for each symbol and store in a dictionary
dataframes = {}
for symbol in symbols:
    fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
    df = fetcher.fetch_candles_as_df()
    df['symbol'] = symbol
    dataframes[symbol] = df

# # Print the DataFrame for each symbol
# for symbol, df in dataframes.items():
#     print(f"\nDataFrame for {symbol}:")
#     print(df)
dataframes.keys()



INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 389
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 389


dict_keys(['ETH-USDT', 'BTC-USDT'])

In [3]:
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma,short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators

).from_apply_func(custom_trading_strategy)




In [4]:
# Entry parameter 
stretch = 0.5
long_term_ma_len = 200
adx_len = 5
atr_len = 3

# exit parameter
short_term_ma_len = 20
df = dataframes[symbols[0]]

# Usage example (commented out)
indicator = custom_strategy_indicator.run(
    df['close'], df['high'], df['low'],
    stretch=stretch,
    long_term_ma_len=long_term_ma_len,
    short_term_ma_len=short_term_ma_len,
    adx_len=adx_len,
    atr_len=atr_len,
    param_product=True

)

indicator.limit_order_price


timestamp
2024-01-09 00:00:00+00:00   NaN
2024-01-10 00:00:00+00:00   NaN
2024-01-11 00:00:00+00:00   NaN
2024-01-12 00:00:00+00:00   NaN
2024-01-13 00:00:00+00:00   NaN
                             ..
2025-01-27 00:00:00+00:00   NaN
2025-01-28 00:00:00+00:00   NaN
2025-01-29 00:00:00+00:00   NaN
2025-01-30 00:00:00+00:00   NaN
2025-01-31 00:00:00+00:00   NaN
Name: (0.5, 200, 20, 5, 3), Length: 389, dtype: float64

In [8]:

# Create arrays to store the data we want to plot
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1] > 0) and (low[c.i] < entries[c.i-1]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder


In [9]:

# candle data
close = df['close']
open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

# indicator data
entries = indicator.limit_order_price.to_numpy()
ma_short = indicator.short_term_ma.to_numpy()

# Create arrays to store data
entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

# Create portfolio with trade_data
pf = vbt.Portfolio.from_order_func(
    close,
    order_func_nb,
    high,
    low,
    open_,
    entries,
    ma_short,
    entry_price,  # Pass entry_price array to the function
    init_cash=500
)

In [14]:
pf.plot().show()

In [17]:
trades = pf.trades.records_readable

In [18]:
check_ma_200 = ta.ema(df['close'], length=200)
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='candlesticks'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.long_term_ma, mode='lines',marker=dict(color='yellow',size=20), name='200 MA'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.short_term_ma, mode='lines',marker=dict(color='red',size=20), name='20 MA'))

# Add annotations for trades
for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Entry Timestamp'],
        y=row['Avg Entry Price'],
        text="Entry",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Exit Timestamp'],
        y=row['Avg Exit Price'],
        text="Exit",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=40
    )
# fig.add_trad(go.Scatter(x=df.index, y=entry_exit_bars['entry_time'],))
# Enable y-axis zooming
fig.update_layout(
    yaxis=dict(fixedrange=False),  # Allow y-axis zooming
    clickmode='event+select'  # Enable click events and selection events

)
